## データセットについて
本モデルの学習には、以下の Kaggle データセットを使用しました。

- **Cat Breeds Dataset**  
  https://www.kaggle.com/datasets/ma7555/cat-breeds-dataset

※ データセットは学習前にダウンロードし、必要な品種のみを  
　`data/raw/` → `data/train/` のディレクトリ構成へ整理しています。  
　この Notebook では、学習プロセスを示すために必要なコードのみ掲載しています。


## データ準備
Kaggle から取得した元データを `data/raw/` に配置し、  
学習で使用する 7 品種を `data/train/` にコピーします。


In [ ]:
import os
import shutil

src_dir = "data/raw"     # 元データ（手動配置）
dst_dir = "data/train"   # 学習に使うフォルダ
os.makedirs(dst_dir, exist_ok=True)

selected_breeds = [
    "Scottish Fold",
    "American Shorthair",
    "Russian Blue",
    "Siamese",
    "Persian",
    "Maine Coon",
    "Norwegian Forest Cat"
]

for breed in selected_breeds:
    src_path = os.path.join(src_dir, breed)
    dst_path = os.path.join(dst_dir, breed)
    if os.path.exists(src_path):
        shutil.copytree(src_path, dst_path, dirs_exist_ok=True)
        print(f"{breed} をコピーしました")
    else:
        print(f"{breed} は元データに存在しません")


## データ読み込み
`data/train/` から画像データを読み込み、前処理を行います。

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

train_dataset = datasets.ImageFolder(root="data/train", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

print(f"画像枚数: {len(train_dataset)}")
print(f"クラス一覧: {train_dataset.classes}")

## モデル定義
ResNet18 の最終層を、分類対象の品種数に合わせて置き換えます。

In [ ]:
import torch.nn as nn
from torchvision import models

model = models.resnet18(pretrained=True)

num_classes = len(train_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

print(model)

## 損失関数と最適化手法
クロスエントロピー損失と Adam を使用します。

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

print("損失関数:", criterion)
print("最適化手法:", optimizer)

## 学習（1 Epoch）
まずは 1 エポックだけ実行して学習が動作するか確認します。

In [ ]:
num_epochs = 1
model.train()

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()           # 勾配リセット
        outputs = model(images)         # 推論
        loss = criterion(outputs, labels)  # 損失計算
        loss.backward()                 # 逆伝播
        optimizer.step()                # 重み更新

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


## 検証データの分割と精度確認
学習データを 8:2 に分割し、検証精度を測定します。

In [ ]:
from torch.utils.data import random_split

train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size

train_subset, val_subset = random_split(train_dataset, [train_size, val_size])
val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)

model.eval()
correct = total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"検証データ精度: {accuracy:.2f}%")


## 追加学習（5 Epoch）
学習を続け、損失と精度の推移を確認します。

In [ ]:
# エポックを増やして精度推移を見る
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # 検証
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {running_loss/len(train_loader):.4f}  Val Acc: {acc:.2f}%")


## モデル保存
学習済みモデル（state_dict）を `model/cat_model.pth` に保存します。

In [ ]:
save_path = "model/cat_model.pth"
os.makedirs("model", exist_ok=True)

torch.save(model.state_dict(), save_path)
print(f"モデルを保存しました: {save_path}")